This notebook is sourced from ch4 Deep learning with Pytorch

In [ ]:
input_path = '../input/bike-sharing-dataset'

In [ ]:
import numpy as np

In [ ]:
bikes_numpy = np.loadtxt("../input/bike-sharing-dataset/hour.csv", 
                         dtype=np.float32,
                         delimiter=",",
                         skiprows=1,
                         converters={1: lambda x:float(x[8:10])})


In [ ]:
import torch

In [ ]:
bikes = torch.from_numpy(bikes_numpy)

In [ ]:
bikes

In [ ]:
import pandas as pd

In [ ]:
bikes_df = pd.read_csv('../input/bike-sharing-dataset/hour.csv')

In [ ]:
bikes_df.head()

In a time series dataset such as this one, rows represent successive time-points: there is
a dimension along which they are ordered. Sure, we could treat each row as independent and try to predict the number of circulating bikes based on, say, a particular time
of day regardless of what happened earlier. However, the existence of an ordering
gives us the opportunity to exploit causal relationships across time. For instance, it
allows us to predict bike rides at one time based on the fact that it was raining at an
earlier time. For the time being, we’re going to focus on learning how to turn our
bike-sharing dataset into something that our neural network will be able to ingest in
fixed-size chunks.

In [ ]:
len(bikes_df[:-3])

# Shaping the data by time period



We might want to break up the two-year dataset into wider observation periods, like
days. This way we’ll have N (for number of samples) collections of C sequences of length
L. In other words, our time series dataset would be a tensor of dimension 3 and shape
N × C × L. The C would remain our 17 channels, while L would be 24: 1 per hour of
the day. There’s no particular reason why we must use chunks of 24 hours, though the
general daily rhythm is likely to give us patterns we can exploit for predictions. We
could also use 7 × 24 = 168 hour blocks to chunk by week instead, if we desired. All of
this depends, naturally, on our dataset having the right size—the number of rows must
be a multiple of 24 or 168. Also, for this to make sense, we cannot have gaps in the
time series

 Let’s go back to our bike-sharing dataset. The first column is the index (the global
ordering of the data), the second is the date, and the sixth is the time of day. We have
everything we need to create a dataset of daily sequences of ride counts and other
exogenous variables. Our dataset is already sorted, but if it were not, we could use
torch.sort on it to order it appropriately

note : the version of file originally used was  hour fixed version not the one we would be using 

In [ ]:
bikes.shape, bikes.stride()

thats 17379 hours and 17 columns.

choosing till bikes[:-3] since 17376 is devided by 24

In [ ]:
daily_bikes = bikes[:-3].view(-1, 24, bikes[:-3].shape[1])
daily_bikes.shape, daily_bikes.stride()

calling view on a tensor returns a new tensor that changes the number of dimensions and the striding information, without
changing the storage. This means we can rearrange our tensor at basically zero cost,
because no data will be copied. Our call to view requires us to provide the new shape
for the returned tensor. We use -1 as a placeholder for “however many indexes are
left, given the other dimensions and the original number of elements.

For daily_bikes, the stride is telling us that advancing by 1 along the hour dimension (the second dimension) requires us to advance by 17 places in the storage (or
one set of columns); whereas advancing along the day dimension (the first dimension) requires us to advance by a number of elements equal to the length of a row in
the storage times 24 (here, 408, which is 17 × 24).
 We see that the rightmost dimension is the number of columns in the original
dataset. Then, in the middle dimension, we have time, split into chunks of 24 sequential hours. In other words, we now have N sequences of L hours in a day, for C channels. To get to our desired N × C × L ordering, we need to transpose the tensor:

In [ ]:
daily_bikes = daily_bikes.transpose(1,2)
daily_bikes.shape, daily_bikes.stride()

lets do something about the weather situation,if we turn it into categorical then we can turn it into a one hot encoded vector

In [ ]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:, 9]

Now we scatter ones into our matrix according to the corresponding level at each row, we need to use unsqueeze for this

In [ ]:
weather_onehot.scatter_(
    dim=1,
    index=first_day[:,9].unsqueeze(1).long() -1,
    value=1.0
)

Our day started with weather “1” and ended with “2,” so that seems right.
Last, we concatenate our matrix to our original dataset using the cat function.
Let’s look at the first of our results

In [ ]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

Here we prescribed our original bikes dataset and our one-hot-encoded “weather situation” matrix to be concatenated along the column dimension (that is, 1). In other
words, the columns of the two datasets are stacked together; or, equivalently, the new
one-hot-encoded columns are appended to the original dataset. For cat to succeed, it
is required that the tensors have the same size along the other dimensions—the row
dimension, in this case. Note that our new last four columns are 1, 0, 0, 0, exactly
as we would expect with a weather value of 1.
We could have done the same with the reshaped daily_bikes tensor. Remember
that it is shaped (B, C, L), where L = 24. We first create the zero tensor, with the same
B and L, but with the number of additional columns as C:

In [ ]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape

Then we scatter the one-hot encoding into the tensor in the C dimension. Since this
operation is performed in place, only the content of the tensor will change:

In [ ]:
daily_weather_onehot.scatter_(
    1, daily_bikes[:,9,:].long().unsqueeze(1) -1 , 1.0
)
daily_weather_onehot.shape

concatenate along C dimension

In [ ]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

We mentioned earlier that this is not the only way to treat our “weather situation” variable. Indeed, its labels have an ordinal relationship, so we could pretend they are special values of a continuous variable. We could just transform the variable so that it runs
from 0.0 to 1.0:

In [ ]:
daily_bikes[:,9,:] = (daily_bikes[:,9,:] - 1.0)/3.0

As we mentioned in the previous section, rescaling variables to the [0.0, 1.0] interval
or the [-1.0, 1.0] interval is something we’ll want to do for all quantitative variables,
like temperature (column 10 in our dataset). We’ll see why later; for now, let’s just say
that this is beneficial to the training process.

There are multiple possibilities for rescaling variables. We can either map their
range to [0.0, 1.0]

In [ ]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)
/ (temp_max - temp_min))

or subtract the mean and divide by the standard deviation:


In [ ]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))
/ torch.std(temp))


In the latter case, our variable will have 0 mean and unitary standard deviation. If our
variable were drawn from a Gaussian distribution, 68% of the samples would sit in the
[-1.0, 1.0] interval.

Great: we’ve built another nice dataset, and we’ve seen how to deal with time series
data. For this tour d’horizon, it’s important only that we got an idea of how a time
series is laid out and how we can wrangle the data in a form that a network will digest